In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils.utils import *
from utils.ml_utils import *
from typing import List, Dict
from pyod.models.ecod import ECOD
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest

In [2]:
file_path = "Data/metrics_collector.tsv"

column_names = [
    'account_id', 'name', 'point', 'call_count', 'total_call_time', 
    'total_exclusive_time', 'min_call_time', 'max_call_time', 
    'sum_of_squares', 'instances', 'language', 'app_name', 
    'app_id', 'scope', 'host', 'display_host', 'pid', 
    'agent_version', 'labels'
]

data = pd.read_csv(file_path, sep='\t', header=None)
data.columns = column_names
metrics_table = make_table(data)
metrics_table

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_87205/471163818.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t', header=None)
/Users/alexander/Учёба/ХАКАТОНЫ/RedLab/redlab_hack/utils/utils.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["point"] = pd.to_datetime(filtered_data["point"])
/Users/alexander/Учёба/ХАКАТОНЫ/RedLab/redlab_hack/utils/utils.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

,time,time_numeric,web_response,throughput,apdex,error
0,2024-04-15 23:32:00,28553732.0,0.007722,3898.0,0.999679,0.000128
1,2024-04-15 23:33:00,28553733.0,0.008363,3917.5,0.999745,0.000128
2,2024-04-15 23:34:00,28553734.0,0.008709,3993.0,0.999687,0.000000
3,2024-04-15 23:35:00,28553735.0,0.008286,3991.5,0.999812,0.000000
4,2024-04-15 23:36:00,28553736.0,0.010874,3915.5,0.999489,0.000000
...,...,...,...,...,...,...
43267,2024-05-16 00:54:00,28597014.0,0.009245,2560.0,0.998828,0.000000
43268,2024-05-16 00:55:00,28597015.0,0.010571,2491.0,0.998495,0.000000
43269,2024-05-16 00:56:00,28597016.0,0.011785,2429.0,0.997736,0.000206
43270,2024-05-16 00:57:00,28597017.0,0.010940,2373.5,0.997999,0.000211


In [3]:
start_date = pd.to_datetime("2024-04-26 23:32:00")
end_date = pd.to_datetime("2024-04-28 00:58:00")
columns_name = ["web_response", "throughput"]
result = ml(metrics_table, start_date, end_date, columns_name)
result

Threshold:  0.9439040432930517
Threshold:  0.9819023145050355


{'web_response':       labels  probability                time     value
 0          0     0.382163 2024-04-26 23:32:00  0.009514
 1          0     0.389431 2024-04-26 23:33:00  0.009780
 2          0     0.407397 2024-04-26 23:34:00  0.010033
 3          0     0.342105 2024-04-26 23:35:00  0.008357
 4          0     0.344407 2024-04-26 23:36:00  0.007444
 ...      ...          ...                 ...       ...
 1522       0     0.474290 2024-04-28 00:54:00  0.011233
 1523       0     0.480579 2024-04-28 00:55:00  0.011087
 1524       0     0.442479 2024-04-28 00:56:00  0.009095
 1525       0     0.499066 2024-04-28 00:57:00  0.011936
 1526       0     0.468432 2024-04-28 00:58:00  0.010456
 
 [1527 rows x 4 columns],
 'throughput':       labels   probability                time   value
 0          0  4.320100e-11 2024-04-26 23:32:00  4349.0
 1          0  4.320100e-11 2024-04-26 23:33:00  4472.5
 2          0  4.320100e-11 2024-04-26 23:34:00  4275.0
 3          0  4.320100e-11 2024-0

In [4]:
def plot_anomalies(data: Dict[str, pd.DataFrame], feature_name : str):
    
    df = data[feature_name]

    plt.figure(figsize=(20, 8))

    plt.plot(df['time'], df['value'], color='blue', label=f'Значения показателя {feature_name}')
    anomalies = df[df['labels'] == 1]
    plt.scatter(anomalies['time'], anomalies['value'], color='red', s=100, label='Аномалия')
    plt.title(f'Временной ряд с аномалиями показателя {feature_name}')
    plt.xlabel('Временной ряд')
    plt.ylabel(f'{feature_name}')
    plt.legend()
    plt.grid(True)
    plt.show()


In [7]:
plot_anomalies(result, feature_name = "web_response")

TypeError: plot_anomalies() got an unexpected keyword argument 'feature_name'

In [6]:
def plot_anomalies(data, anomaly_mask, value_col):
    
    metrics_table_copy = data.copy()
    metrics_table_copy['anomaly'] = anomaly_mask
    
    plt.figure(figsize=(40, 20))
    plt.plot(metrics_table_copy["time"], metrics_table_copy[value_col], color='blue', label=value_col)
    plt.scatter(metrics_table_copy[metrics_table_copy['anomaly'] == 1]["time"], 
                metrics_table_copy[metrics_table_copy['anomaly'] == 1][value_col], 
                color='red', label='Anomaly', zorder=5)
    plt.title(f'{value_col} with Anomalies')
    plt.xlabel('Time')
    plt.ylabel(value_col)
    plt.legend()
    plt.grid(True)
    plt.show()
